In [1]:
# import pandas as pd
from comfort_lib.src.database.communication import create_connection
from comfort_lib.src.preprocessing.gridding import GridManager, create_wide_table_online, load_wide_table, get_missing_value_info_offline

ModuleNotFoundError: No module named 'comfort_lib'

In [10]:
import pandas as pd
import numpy as np
import sqlite3

In [4]:
output_path = "output/imputation/grid_description/"

In [5]:
# plotting utilities
map_column_label = {"P_TEMPERATURE": "Temperature", "P_SALINITY": "Salinity", "P_PHOSPHATE": "Phosphate", "P_OXYGEN": "Oxygen", 
                   "P_NITRATE": "Nitrate", "P_SILICATE": "Silicate", "LATITUDE": "Latitude", "LONGITUDE": "Longitude", 
                    "LEV_M": "Depth"}
map_column_unit = {"P_TEMPERATURE": "°C", "P_SALINITY": "psu", "P_OXYGEN": "µmol/kg", "P_PHOSPHATE": "µmol/kg", 
                   "P_NITRATE": "µmol/kg", "P_SILICATE": "µmol/kg", "LATITUDE": "", "LONGITUDE": "", 
                    "LEV_M": "m"}

In [15]:
# grid configuration
depth_levels = [0, 50, 100, 200, 300, 400, 500, 1000, 1500, 2000, 3000, 4000, 5000]
grid_config = {"param_tables": ["P_TEMPERATURE", "P_SALINITY", "P_OXYGEN", "P_NITRATE", "P_SILICATE", "P_PHOSPHATE"],
               "lat_min": 0,
               "lat_max": 70,
               "dlat": 1,
               "lon_min": -77,
               "lon_max": 30,
               "dlon": 1,
               "z_min": None,
               "z_max": None,
               "dz": None,
               "z_array": np.array(depth_levels),
               "time_min": "1772-01-01 00:00:00",
               "time_max": "2020-07-08 04:45:00",
               "mode": "Y",
               "selection": None,
               "dtime": 300,
               "note": "northern Atlantic, all times, larger depth steps, more params, exclude Pacific"
             }

param_tables = grid_config["param_tables"]
lat_min = grid_config["lat_min"]
lat_max = grid_config["lat_max"]
dlat = grid_config["dlat"]
lon_min = grid_config["lon_min"]
lon_max = grid_config["lon_max"]
dlon = grid_config["dlon"]
z_min = grid_config["z_min"]
z_max = grid_config["z_max"]
dz = grid_config["dz"]
z_array = np.array(grid_config["z_array"])
time_min = grid_config["time_min"]
time_max = grid_config["time_max"]
mode = grid_config["mode"]
selection = grid_config["selection"]
dtime = grid_config["dtime"]

In [12]:
# database connection
db_path = "C:/users/yvjennig/pycharmprojects/data/6_comfort_potT.sqlite"
conn = sqlite3.connect(db_path)

In [13]:
# define bathymetry parameters
bathymetry_grid_path = "../../data/bathymetry/gebco_2022_sub_ice_topo/gebco_2022_sub_ice_topo.nc"
lat_variable = "lat"
lon_variable = "lon"
depth_variable = "elevation"

# Raw data exploration

In [16]:
include_z_max = True
z_eq = "<=" if include_z_max else "<"
z_min = min(z_array)
z_max = max(z_array)

def load_param_table(connection, table):
    q = f"select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL " \
    f"from {table} where " \
    f"LATITUDE >= {lat_min} and LATITUDE <= {lat_max} and " \
    f"LONGITUDE >= {lon_min} and LONGITUDE <= {lon_max} and " \
    f"LEV_M >= {z_min} and LEV_M {z_eq} {z_max} and " \
    f"DATEANDTIME >= '{time_min}' and DATEANDTIME <= '{time_max}' " \
    f";"
    print(q)   
    cur = connection.execute(q)
    temp = pd.DataFrame(cur.fetchall(), columns=[x[0] for x in cur.description])
    return temp

In [17]:
nitrate = load_param_table(conn, "P_NITRATE")
silicate = load_param_table(conn, "P_SILICATE")
phosphate = load_param_table(conn, "P_PHOSPHATE")
temperature = load_param_table(conn, "P_TEMPERATURE")
salinity = load_param_table(conn, "P_SALINITY")
oxygen = load_param_table(conn, "P_OXYGEN")

select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL from P_NITRATE where LATITUDE >= 0 and LATITUDE <= 70 and LONGITUDE >= -77 and LONGITUDE <= 30 and LEV_M >= 0 and LEV_M <= 5000 and DATEANDTIME >= '1772-01-01 00:00:00' and DATEANDTIME <= '2020-07-08 04:45:00' ;
select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL from P_SILICATE where LATITUDE >= 0 and LATITUDE <= 70 and LONGITUDE >= -77 and LONGITUDE <= 30 and LEV_M >= 0 and LEV_M <= 5000 and DATEANDTIME >= '1772-01-01 00:00:00' and DATEANDTIME <= '2020-07-08 04:45:00' ;
select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL from P_PHOSPHATE where LATITUDE >= 0 and LATITUDE <= 70 and LONGITUDE >= -77 and LONGITUDE <= 30 and LEV_M >= 0 and LEV_M <= 5000 and DATEANDTIME >= '1772-01-01 00:00:00' and DATEANDTIME <= '2020-07-08 04:45:00' ;
select LATITUDE, LONGITUDE, LEV_M, DATEANDTIME, VAL from P_TEMPERATURE where LATITUDE >= 0 and LATITUDE <= 70 and LONGITUDE >= -77 and LONGITUDE <= 30 and LEV_M >= 0 and LEV_M <= 5000 and DATEANDTIME >= '1

KeyboardInterrupt: 

# Define the grid

This requires the data from the sqlite database.

In [ ]:
# apply gridding
grid_manager = GridManager(db_path=db_path, grid_info_table="grid_info")
grid = grid_manager.create_grid(lat_min=lat_min, lat_max=lat_max, dlat=dlat,
                                   lon_min=lon_min, lon_max=lon_max, dlon=dlon,
                                   z_min=z_min, z_max=z_max, dz=dz, z_array=z_array,
                                   time_min=time_min, time_max=time_max, dtime=dtime, mode=mode,
                                   selection=selection,
                                   bathymetry_grid_path=bathymetry_grid_path,
                                   lat_variable=lat_variable, lon_variable=lon_variable,
                                   depth_variable=depth_variable
                                   )

In [ ]:
# apply mapping and wide table creation
mapped = grid.map_tables(connection=conn, param_tables=param_tables, using_database=True, include_z_max=True)
wide_table_name = create_wide_table_online(connection=conn, grid_id=grid.grid_id, param_tables=param_tables)
df_wide = load_wide_table(connection=conn, wide_table_name=wide_table_name, dropping_land_cells=True)
num_nulls = get_missing_value_info_offline(df_wide)
print(num_nulls)

In [ ]:
print(wide_table_name)
print(mapped)

Salinity, temperature and oxygen have a good coverage with less than 10% missing values. The three nutrients contain more empty cells, the worst being nitrate with 35%. 

In [ ]:
plot_missing_value_info(num_nulls, save_as=output_path + "missing_value_proportion.png")